In [13]:
import json
from datetime import datetime
from tqdm import tqdm
from langchain.prompts import ChatPromptTemplate
from shroom_classifier_v13 import ShroomClassifier

In [3]:
DATASET = json.load(open('reference/val.model-agnostic.json', 'r'))
MODEL = "gpt-4-0125-preview"
TEMP = 1.2
CLASSIFIER = ShroomClassifier(model_name=MODEL, temperature=TEMP, example_selection=4)

In [8]:
datapoint = DATASET[0]
submission = {
    "task": CLASSIFIER.TASKS[datapoint["task"]], 
    "role": CLASSIFIER.ROLES[datapoint["task"]],
    "definition": CLASSIFIER.DEFINITION, 
    "examples": CLASSIFIER.examples[datapoint["task"]],
    "src": datapoint["src"], 
    "tgt": datapoint["tgt"], 
    "hyp": datapoint["hyp"],
}

In [12]:
CLASSIFIER.PROMPT

"A language model has generated an output from a given input for a specific task.\n{task}\n{role}\nYou will be given three inputs: input text, target text, and generated text.\nYou are asked to evaluate the generated text looking at the input text and the target text. \nThen, you need to answer the question: is the generated text a hallucination or not? \n{definition}\nAnswer 'Hallucination' if the output is a hallucination, or 'Not Hallucination' if it is not a hallucination. \nOnly answer 'Hallucination' or 'Not Hallucination'.\n{examples}\n##\nInput text: {src}\nTarget text: {tgt}\nGenerated text: {hyp}\nAnswer:\n"

In [21]:
template = ChatPromptTemplate.from_template(CLASSIFIER.PROMPT)
prompt_str = template.format(**submission)

In [23]:
print(prompt_str)

Human: A language model has generated an output from a given input for a specific task.
The given task is Definition Modeling, meaning that the goal of the language model is to generate a definition for a specific term in the input text.
You are a lexicographer concerned that the generated text accurately captures the meaning of the term between the '<define>' and '</define>' delimiters in the input text.
You will be given three inputs: input text, target text, and generated text.
You are asked to evaluate the generated text looking at the input text and the target text. 
Then, you need to answer the question: is the generated text a hallucination or not? 
A text contains a hallucination if and only if it contains any nonsensical or 
factually incorrect information, or contains any additional information that cannot be supported by either 
the input text or the target text.

Answer 'Hallucination' if the output is a hallucination, or 'Not Hallucination' if it is not a hallucination. 
O